In [3]:
#!pip install folium

In [4]:
from folium import Map, TileLayer
import json
from matplotlib import cm
import requests

# Discover data from STAC

In [5]:
stac_endpoint = "https://stac.maap-project.org"
titiler_endpoint = "https://titiler.ops.maap-project.org"
collection = "AfriSAR_AGB_Maps_1681"
r = requests.get(f"{stac_endpoint}/collections/{collection}/items").json()
url = r['features'][0]['assets']['data']['href']
url

's3://nasa-maap-data-store/file-staging/nasa-map/AfriSAR_AGB_Maps_1681___1/Rabi_AGB_50m.tif'

# Get data values from the `/statistics` endpoint

In [6]:
# You can use gdalinfo /vsis3/nasa-maap-data-store/file-staging/nasa-map/SRTMGL1_COD___001/N21W089.SRTMGL1.tif -stats
# or you can get metadata from titiler.
r = requests.get(
    f"{titiler_endpoint}/cog/statistics",
    params = {
        "url": url
    }
).json()

In [7]:
buckets = r['b1']['histogram'][1]
bucket_ranges = [[buckets[i],buckets[i+1]] for i in range(len(buckets)-1)]
bucket_ranges

[[0.08733899891376495, 51.720603942871094],
 [51.720603942871094, 103.35386657714844],
 [103.35386657714844, 154.9871368408203],
 [154.9871368408203, 206.62039184570312],
 [206.62039184570312, 258.253662109375],
 [258.253662109375, 309.8869323730469],
 [309.8869323730469, 361.52020263671875],
 [361.52020263671875, 413.1534423828125],
 [413.1534423828125, 464.7867126464844],
 [464.7867126464844, 516.4199829101562]]

# Pick a color map and create a linear mapping

In [8]:
# All colormaps can be found here: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
sequential_cmaps = [
    'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
    'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
    'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'
]

sequential_cmaps2 = [
    'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
    'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
    'hot', 'afmhot', 'gist_heat', 'copper']

In [9]:
rgbs = [[int(value) for value in rgb] for rgb in cm.ScalarMappable(cmap='gist_earth').to_rgba(x=buckets[:-1], bytes=True)]
color_map = list(zip(bucket_ranges, rgbs))
# some tweaking may be necessary
color_map

[([0.08733899891376495, 51.720603942871094], [0, 0, 0, 255]),
 ([51.720603942871094, 103.35386657714844], [18, 48, 119, 255]),
 ([103.35386657714844, 154.9871368408203], [37, 102, 124, 255]),
 ([154.9871368408203, 206.62039184570312], [54, 135, 111, 255]),
 ([206.62039184570312, 258.253662109375], [67, 151, 77, 255]),
 ([258.253662109375, 309.8869323730469], [123, 167, 82, 255]),
 ([309.8869323730469, 361.52020263671875], [169, 179, 91, 255]),
 ([361.52020263671875, 413.1534423828125], [191, 164, 100, 255]),
 ([413.1534423828125, 464.7867126464844], [221, 186, 167, 255]),
 ([464.7867126464844, 516.4199829101562], [253, 250, 250, 255])]

# Preview the data

In [10]:
cmap = json.dumps(color_map)

r = requests.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "colormap": cmap
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"] + 1
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Swisstopo"
)
aod_layer.add_to(m)
m